In [ ]:
import json
import os
import re

try:
    import speech_recognition as sr
except ModuleNotFoundError:
    print("Please install 'SpeechRecognition' using pip.")

try:
    from pdfminer.high_level import extract_text
except ModuleNotFoundError:
    print("Please install 'pdfminer.six' using pip.")

try:
    from pptx import Presentation
except ModuleNotFoundError:
    print("Please install 'python-pptx' using pip.")

try:
    from docx import Document
except ModuleNotFoundError:
    print("Please install 'python-docx' using pip.")

try:
    from PIL import Image
except ModuleNotFoundError:
    print("Please install 'Pillow' using pip.")

try:
    import pytesseract
except ModuleNotFoundError:
    print("Please install 'pytesseract' using pip.")

# Transcribe voice recording and format as a conversation
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(file_path) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)

            # Simulating conversation extraction (for demo purposes, split text by sentence)
            conversation = re.split(r'(?<=[.!?]) +', text)
            json_output = {}
            for idx, sentence in enumerate(conversation):
                user = f"user{(idx % 2) + 1}"
                json_output[user] = json_output.get(user, "") + sentence + " "

            return json.dumps(json_output, ensure_ascii=False, indent=4)
    except sr.UnknownValueError:
        return "Speech Recognition could not understand the audio."
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"
    except Exception as e:
        return f"Error transcribing audio: {e}"

# Extract structured data (e.g., Name, Details, Cost) from PDFs
def extract_data_from_pdf(file_path):
    try:
        text = extract_text(file_path)

        # Simulate extracting specific fields (example assumes it's an invoice)
        name = re.search(r"Name: (\w+)", text)
        details = re.search(r"Details: ([\w\s]+)", text)
        cost = re.search(r"Cost: (\d+\.\d{2})", text)

        data = {
            "Name": name.group(1) if name else "N/A",
            "Details": details.group(1) if details else "N/A",
            "Cost": cost.group(1) if cost else "N/A"
        }
        return json.dumps(data, ensure_ascii=False, indent=4)
    except Exception as e:
        return f"Error reading PDF: {e}"

# Extract text from PPT files
def extract_data_from_ppt(file_path):
    try:
        ppt = Presentation(file_path)
        text = ""
        for slide in ppt.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text += shape.text + "\n"
        return json.dumps({"data": text}, ensure_ascii=False, indent=4)
    except Exception as e:
        return f"Error reading PowerPoint: {e}"

# Extract text from Word documents
def extract_data_from_docx(file_path):
    try:
        doc = Document(file_path)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return json.dumps({"data": text}, ensure_ascii=False, indent=4)
    except Exception as e:
        return f"Error reading Word document: {e}"

# Extract text from images using OCR
def extract_text_from_image(file_path):
    try:
        image = Image.open(file_path)
        text = pytesseract.image_to_string(image)
        return json.dumps({"data": text}, ensure_ascii=False, indent=4)
    except Exception as e:
        return f"Error reading image: {e}"

# Main function to determine file type and extract data accordingly
def extract_data(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension == '.pdf':
        return extract_data_from_pdf(file_path)
    elif file_extension == '.pptx':
        return extract_data_from_ppt(file_path)
    elif file_extension == '.docx':
        return extract_data_from_docx(file_path)
    elif file_extension in ['.jpg', '.jpeg', '.png']:
        return extract_text_from_image(file_path)
    elif file_extension in ['.mp3', '.wav']:
        return transcribe_audio(file_path)
    else:
        return json.dumps({"error": "Unsupported file format."}, ensure_ascii=False, indent=4)

# Generate JSON output
def generate_json(file_path):
    return extract_data(file_path)

if __name__ == "__main__":
    file_path = input("Enter the path of the file: ")
    if os.path.isfile(file_path):
        json_output = generate_json(file_path)
        print(json_output)
    else:
        print(f"File not found: {file_path}")


Please install 'SpeechRecognition' using pip.
Please install 'pdfminer.six' using pip.
Please install 'python-pptx' using pip.
Please install 'python-docx' using pip.
Please install 'pytesseract' using pip.
